In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from firebase_admin.firestore import FieldFilter
from google.cloud.firestore_v1.document import DocumentReference
from datetime import datetime
import json 
from dotenv import load_dotenv
from random import randint
import os
import re
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'    
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


def load_json_file(filename):
    with open(filename) as fp:
        content = json.load(fp)
    return content


In [34]:
DB = {
    "volumes": {},
    "items": {},
    "usuarios": {},
    "ambientes": {},
    "ambientes-norte": {}
}

In [18]:
[ x for x in dir(DocumentReference) if not x.startswith("__") ]

['_document_path',
 '_document_path_internal',
 '_prep_batch_get',
 '_prep_collections',
 '_prep_create',
 '_prep_delete',
 '_prep_set',
 '_prep_update',
 'collection',
 'collections',
 'create',
 'delete',
 'get',
 'id',
 'on_snapshot',
 'parent',
 'path',
 'set',
 'update']

In [30]:
"e" not in {"a": 1, "e": -3}

False

In [31]:
def split_string_ref(str_fullref: str):
    collection, key = str_fullref.split("/")
    return collection, key

def process_reference(reference: DocumentReference):  
    col, id = split_string_ref(reference.path)
    print(f"{bcolors.OKCYAN}Processing {col}/{id}{bcolors.ENDC}")
    if id not in  DB[col] :
        print("API call to get data")
        DB[col][id] = process_dict(reference.get().to_dict())
    return reference.path
    
def process_array(array: list):
    return [process_reference(item) if type(item) == DocumentReference else item for item in array]

def process_dict(input_dict):  
    output_dict = {}
    for key, value in input_dict.items():
        if type(value) == DocumentReference:
            output_dict[key] = process_reference(value)
        elif isinstance(value, dict):
            output_dict[key] = process_dict(value)
        elif isinstance(value, list):
            output_dict[key] = process_array(value)
        elif isinstance(value, datetime):
            output_dict[key] = value.isoformat()
        else:
            output_dict[key] = value
    return output_dict

In [35]:
CODIGOS_LEMA = ["V353341E" , "V353341I", "B1230028", "V363741", "V347859", "Mp4qqn9Lmzowei87GmSn"]

for chave in  CODIGOS_LEMA:
    doc_ref = db.document(f"volumes/{chave}")
    doc = doc_ref.get()
    if doc.exists:
        print(f"{bcolors.OKGREEN}Documento {chave} encontrado{bcolors.ENDC}")
        DB["volumes"][chave] = {}
        documento = process_dict(doc.to_dict())
        DB["volumes"][chave] = documento
    else:
        print(f"{bcolors.FAIL}Documento {chave} não encontrado{bcolors.ENDC}")

Documento V353341E encontrado
Processing items/353341-E
API call to get data
Processing ambientes/B123
API call to get data
Processing usuarios/l.f.bossa@ufsc.br
API call to get data
Processing volumes/V353341E
Processing usuarios/l.f.bossa@ufsc.br
Processing ambientes/B123
Processing ambientes-norte/A102
API call to get data
Processing ambientes/B123
Documento V353341I encontrado
Processing items/353341-I
API call to get data
Processing ambientes/B123
Processing volumes/V353341I
Processing usuarios/l.f.bossa@ufsc.br
Processing ambientes/B123
Processing ambientes-norte/A102
Processing ambientes/B123
Documento B1230028 encontrado
Processing items/363637
API call to get data
Processing ambientes/B123
Processing volumes/B1230028
Processing usuarios/l.f.bossa@ufsc.br
Processing ambientes/B123
Processing ambientes-norte/A102
Processing ambientes/B123
Documento V363741 encontrado
Processing items/363741
API call to get data
Processing ambientes/B123
Processing volumes/V363741
Processing usua

In [37]:
json.dump(DB, open("volumes-lema.json", "w"), indent=4, ensure_ascii=False)

# Carrega os dados que já estão salvos

In [4]:
items = load_json_file("json1123/itens.json")
volumes = load_json_file("json1123/volumes.json")

In [5]:
def load_data_from_fullref(fullref: str):
    collection, key = split_string_ref(fullref)
    if collection == "volumes":  
        return volumes[key]
    elif collection == "items":
        return items[key]
    

In [6]:
def batch_write(collection: str, dictarray: list[dict], key_id: str | None = None):
    batch = db.batch()
    collection_ref = db.collection(collection)
    k = 1
    for i in range(0, len(dictarray), 500):
        chunk = dictarray[i:i+500]
        for document in chunk:
            if key_id:
                doc_ref = collection_ref.document(str(document[key_id]))
                batch.set(doc_ref, document)
        print(f"{collection}: chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [7]:
def batch_soft_delete(fullrefs_list: list): 
    batch = db.batch() 
    k = 1
    for i in range(0, len(fullrefs_list), 500):
        chunk = fullrefs_list[i:i+500]
        for full_ref in chunk: 
            doc_ref = db.document(full_ref)
            batch.update(doc_ref, {'deleted': True})
        print(f"chunk {k} ({len(chunk)} documents)")
        k += 1
        batch.commit()

In [8]:
import re
import pandas as pd

In [9]:
ares_condicionados = [ x for x in items.values() if re.match(r".*CONDICION.*", x["short_descricao"])]

In [10]:
def extract_btus(descricao: str):
    match = re.search(r".*?(\d*\.{0,1}\d+)\s*BTU.*", descricao)
    if match:
        return match.group(1).replace(".", "")
    else:
        return None

In [11]:
def extract_marca(descricao: str):
    match = re.search(r".*?MARCA:*\s*(\w+).*", descricao)
    if match:
        return match.group(1)
    else:
        return None

In [12]:
def extract_groups(regex: str, descricao: str):
    match = re.search(regex, descricao)
    if match:
        return match.groups()
    else:
        return None

In [13]:
def extract_modelo(descricao: str):
    modelo = extract_groups(r"MODELO(.*?),", descricao)
    if modelo:
        return modelo[0]
    else:
        return None

In [14]:
btus = list(map(lambda x: extract_btus(x["detalhes"]["descricao"]), ares_condicionados))
codigo = [x["key"] for x in ares_condicionados]
short_descricao = [x["short_descricao"] for x in ares_condicionados]
descricao = [x["detalhes"]["descricao"] for x in ares_condicionados]
marca = list(map(lambda x: extract_marca(x["detalhes"]["descricao"]), ares_condicionados))
origem = [x["origem"] for x in ares_condicionados]
modelo = list(map(lambda x: extract_modelo(x["detalhes"]["descricao"]), ares_condicionados))

ares_df = pd.DataFrame({"codigo": codigo, 
                        "short_descricao": short_descricao, 
                        "origem": origem,
                        "descricao": descricao, 
                        "btus": btus, 
                        "marca": marca,
                        "modelo": modelo})

ares_df.btus = ares_df.btus.astype(int)

In [15]:
ares_df

,codigo,short_descricao,origem,descricao,btus,marca,modelo
0,227811,CONDICIONADOR DE AR 9000BTUS,B110,CONDICIONADOR DE AR 9000BTUS - CONDICIONADOR D...,9000,None,None
1,227821,CONDICIONADOR DE AR 9000BTUS,A003,CONDICIONADOR DE AR 9000BTUS - CONDICIONADOR D...,9000,None,None
2,227828,CONDICIONADOR DE AR 9000BTUS,A108,CONDICIONADOR DE AR 9000BTUS - CONDICIONADOR D...,9000,None,None
3,227831,CONDICIONADOR DE AR 9000BTUS,B003.1,CONDICIONADOR DE AR 9000BTUS - CONDICIONADOR D...,9000,None,None
4,227839,CONDICIONADOR DE AR 9000BTUS,B019,CONDICIONADOR DE AR 9000BTUS - CONDICIONADOR D...,9000,None,None
...,...,...,...,...,...,...,...
154,359953,CONDICIONADOR DE AR,B118,CONDICIONADOR DE AR - CONDICIONADOR DE AR SPLI...,24000,GREE,GWC24ME-D1NNA8E
155,363979,CONDICIONADOR DE AR SPLIT INVERTER,B001,CONDICIONADOR DE AR SPLIT INVERTER - HI-WALL C...,9000,ELGIN,: ELGIN HVFI09B2IA NACIONAL. INVERTER; QUENTE ...
156,363980,CONDICIONADOR DE AR SPLIT INVERTER,B119,CONDICIONADOR DE AR SPLIT INVERTER - HI-WALL C...,9000,ELGIN,: ELGIN HVFI09B2IA NACIONAL. INVERTER; QUENTE ...
157,420030,APARELHO DE AR CONDICIONADO,B008,APARELHO DE AR CONDICIONADO - AR CONDICIONADO ...,12000,ELGIN,None


# Tamanho dos ares 

In [16]:
ARES_SIZES = pd.read_excel("xlsx/arescondicionares.xlsx")

In [17]:
ARES_SIZES.groupby(["BTUS", "Unidade"]).agg({"A": 'mean', "L": 'mean', "C": 'mean', "Peso": 'mean'})

A           L          C        Peso
BTUS  Unidade                                           
9000  Externa   52.40   71.400000  26.600000   24.000000
      Interna   27.40   75.200000  19.200000    8.000000
12000 Externa   55.00   66.333333  33.333333   24.666667
      Interna   29.00   83.666667  19.333333    9.000000
18000 Externa   53.25   77.000000  43.750000   31.250000
      Interna   32.00   98.750000  26.750000   11.250000
22000 Externa   70.00   96.000000  40.000000   50.000000
      Interna   34.00  108.000000  23.000000   16.000000
24000 Externa   58.00   96.500000  53.500000   47.000000
      Interna   32.00  106.500000  32.500000   11.500000
60000 Externa  116.00   87.000000  33.000000  101.000000
      Interna   68.00  129.000000  25.000000   49.000000

In [18]:
[x["meta"]["partes"] for x in ares_condicionados if x["meta"]["inteiro"] == False]

[['items/322601-2', 'items/322601-3'], ['items/322607-3', 'items/322607-4']]

In [19]:
def print_link_to_data(fullref: str):
    url = "https://console.firebase.google.com/u/0/project/frete-norte-ufsc-blumenau/firestore/data/"
    print(f"{bcolors.OKBLUE}{url}{fullref}{bcolors.ENDC}")

In [20]:
def match_modelo(modelo_raw: str | None):
    if modelo_raw == None:
        return None
    unique_modelos = ARES_SIZES["Modelo"].unique()
    for modelo in unique_modelos:
        if modelo in modelo_raw:
            return modelo

In [21]:
ares_df["real_modelo"] = ares_df.modelo.apply(match_modelo)

In [22]:
from copy import deepcopy

# Ambientes norte?

In [23]:
ambientes_norte = load_json_file("json0929/ambientes-norte.json")

In [24]:
def find_destino(origem: str):
    for ambiente in ambientes_norte:
        if origem in ambiente["origem"]:
            return ambiente["codigo"]

In [25]:
ares_df["destino"] = ares_df.origem.apply(find_destino)

In [26]:
AMBIENTES = {}

for codigo in ares_df.origem.unique():
    docref = db.document(f"ambientes/{codigo}")
    doc = docref.get().to_dict()
    AMBIENTES[codigo] = doc

In [27]:
AMBIENTES["B110"]

{'codigo': 'B110',
 'edificio': 'BNU03 - Bloco Acadêmico B',
 'items': 15,
 'nome': 'LABCTI - LABORATÓRIO DE CIÊNCIA, TECNOLOGIA E INOVAÇÃO (1º Andar)',
 'status': 'Em uso',
 'campus': 'Sul',
 'lider': <google.cloud.firestore_v1.document.DocumentReference at 0x7fa38265ff10>}

In [28]:
initial = {'a': 1, 'b': 2, 'obj': {'c': 3, 'd': 4}}

second = deepcopy(initial)

second['obj']['c'] = 5

print(initial)

{'a': 1, 'b': 2, 'obj': {'c': 3, 'd': 4}}


In [29]:
def create_part(item_original: dict, part_dict: dict): 
    """part_dict: dict with keys: codigo, descricao, medidas, peso
    """
    item = deepcopy(item_original) 
    item["key"] = f"{item_original['key']}-{part_dict['codigo']}"
    item["tipo"] = "Parte"
    item["short_descricao"] = part_dict["descricao"] 
    item["meta"]["partes"] = []
    item["meta"]["inteiro"] = True
    item["meta"]["volumado"] = False
    item["meta"]["partes"] = []
    item["meta"]["updated"] = datetime.now(tz=datetime.now().astimezone().tzinfo)
    item["detalhes"]["descricao"] = item_original["short_descricao"] + f" - {part_dict['descricao']}"
    item["detalhes"]["medidas"] = metidas_dict_to_str(part_dict["medidas"])
    item["detalhes"]["peso"] = int(part_dict["peso"])
    return item

def metidas_dict_to_str(medidas_dict: dict):
    return f"{medidas_dict['a']} x {medidas_dict['l']} x {medidas_dict['c']} (AxLxC)"

 

In [30]:
def generate_partes_dict(index: int):
    row = ARES_SIZES.iloc[index] 
    medidas_dict = {"a": int(row.A), "l": int(row.L), "c": int(row.C)}
    return {"codigo": row.Unidade[0], "descricao": "Unidade " + row.Unidade, "medidas": medidas_dict, "peso": int(row.Peso)}

In [31]:
def generate_generic_partes_dict(btus):
    groupby = ARES_SIZES[ARES_SIZES.BTUS == btus].groupby("Unidade")
    values = groupby.agg({"A": 'max', "L": 'max', "C": 'max', "Peso": 'max'})
    resultado = values.reset_index()
    for index, row in resultado.iterrows():
        medidas_dict = {"a": int(row.A), "l": int(row.L), "c": int(row.C)}
        yield {"codigo": row.Unidade[0], "descricao": "Unidade " + row.Unidade, "medidas": medidas_dict, "peso": int(row.Peso)}

In [32]:
list(generate_generic_partes_dict(9000))

[{'codigo': 'E',
  'descricao': 'Unidade Externa',
  'medidas': {'a': 55, 'l': 78, 'c': 32},
  'peso': 29},
 {'codigo': 'I',
  'descricao': 'Unidade Interna',
  'medidas': {'a': 29, 'l': 86, 'c': 20},
  'peso': 10}]

In [33]:
def find_ar_data(ar_row: pd.Series):
    btus_mask = ARES_SIZES["BTUS"] == ar_row["btus"]
    marca_mask = ARES_SIZES["Marca"] == ar_row["marca"]
    modelo_mask = ARES_SIZES["Modelo"] == ar_row["real_modelo"]
    mask = btus_mask & marca_mask & modelo_mask
    return ARES_SIZES[mask]


In [34]:
from datetime import datetime

def create_volume(item_dict: dict , parte_dict: dict):
    lider = AMBIENTES[item_dict['origem']]['lider']
    destino_str = find_destino(item_dict['origem'])
    destino = db.document(f"ambientes-norte/{destino_str}")
    itemref = db.document(f"items/{item_dict['key']}")
    volume_model = {'medidas': parte_dict['medidas'],
        'categoria': 'Ar Condicionado',
        'peso': parte_dict['peso'],
        'items': [itemref],
        'observacao': 'Criado pelo sistema', 
        'data_criacao': datetime.now(),
        'embalagem': {'caixa': None,
        'platico_bolha': None,
        'enchimento': True},
        'responsavel': lider,
        'origem': item_dict['ambiente'],
        'destino': destino,
        'propriedades': ['Frágil', 'Proibido Empilhar'],
        'codigo': f"V{item_dict['key']}".replace("-", ""),
        'servicos': ['Montagem e Desmontagem', 'Especializados'],
        'status': ['Criado', 'Para Desmontagem'],
        'localizacao_atual':  item_dict['ambiente'],
        'deleted': False}    
    return volume_model

In [54]:
def process_data(data_string):
    try:
        return datetime.fromisoformat(data_string, tz=datetime.now().astimezone().tzinfo)
    except:
        return None


def process_item(item: dict): 
    copia = deepcopy(item) 
    copia["meta"]["inteiro"] = False
    copia["meta"]["volumado"] = False
    copia["meta"]["volume"] = None
    copia["meta"]["updated"] = datetime.now(tz=datetime.now().astimezone().tzinfo)
    copia["ambiente"] = db.document(f"ambientes/{copia['origem']}")
    copia["detalhes"]["incorporacao"] = process_data(item["detalhes"]["incorporacao"]) 
    copia["detalhes"]["transf_local"] = process_data(item["detalhes"]["transf_local"]) 
    codigo = item["key"]
    thisindex = ares_df[ ares_df.codigo == codigo].index
    row = ares_df.iloc[thisindex[0],:]
    partes_dict = [generate_partes_dict(x) for x in find_ar_data(row).index]
    if len(partes_dict) == 2:
        pass # tudo certo
    else:
        partes_dict = list(generate_generic_partes_dict(row.btus))
    novas_partes = [create_part(copia, x) for x in partes_dict]
    copia["meta"]["partes"] = [db.document(f"items/{x['key']}") for x in novas_partes]
    volumes = [create_volume(x, y) for (x,y) in zip(novas_partes,partes_dict)]
    return copia, novas_partes, volumes

In [55]:
process_item(ares_condicionados[15])

({'origem': 'A101',
  'edificio': 'BNU02 - Bloco Acadêmico A',
  'short_descricao': 'CONDICIONADOR DE AR SPLIT INVERTER',
  'key': 322569,
  'tipo': 'Permanente',
  'ambiente': <google.cloud.firestore_v1.document.DocumentReference at 0x7fa3802930a0>,
  'meta': {'volume': None,
   'updated': datetime.datetime(2023, 12, 14, 16, 8, 8, 348890, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=75600), '-03')),
   'inteiro': False,
   'volumado': False,
   'partes': [<google.cloud.firestore_v1.document.DocumentReference at 0x7fa38027bd90>,
    <google.cloud.firestore_v1.document.DocumentReference at 0x7fa3802936a0>]},
  'detalhes': {'patrimonio': 510164,
   'medidas': '',
   'valor': 1039.05,
   'n_controle': nan,
   'descricao': 'CONDICIONADOR DE AR SPLIT INVERTER - HI-WALL COM ROTAÇÃO VARIÁVEL \x96 CAPACIDADE 11.500 A 12.000 BTU/H, FRIO, COM INSTALAÇÃO INCLUSA - CONDICIONADOR DE AR SPLIT, 12.000 BTUS, MARCA ELECTROLUX, MODELO BI12F, FRIO, COR BRANCA, 220V. ACOMPANHA CONTROLE REM

In [37]:
from google.cloud.firestore_v1.document import DocumentReference

def process_reference(reference: DocumentReference):
    return reference.path

def process_array(array: list):
    return [process_reference(item) if type(item) == DocumentReference else item for item in array]

def process_dict(input_dict):
    output_dict = {}
    for key, value in input_dict.items():
        if type(value) == DocumentReference:
            output_dict[key] = process_reference(value)
        elif isinstance(value, dict):
            output_dict[key] = process_dict(value)
        elif isinstance(value, list):
            output_dict[key] = process_array(value)
        elif isinstance(value, datetime):
            output_dict[key] = value.isoformat()
        else:
            output_dict[key] = value
    return output_dict

In [38]:
[x for x in ares_condicionados if x["key"] == 322563]

[{'origem': 'B117',
  'edificio': 'BNU03 - Bloco Acadêmico B',
  'short_descricao': 'CONDICIONADOR DE AR SPLIT INVERTER',
  'key': 322563,
  'tipo': 'Permanente',
  'ambiente': 'ambientes/B117',
  'meta': {'volume': 'volumes/B1170017',
   'updated': '2023-09-14T11:40:32+00:00',
   'inteiro': True,
   'volumado': True,
   'partes': []},
  'detalhes': {'patrimonio': 510158,
   'medidas': '',
   'valor': 1039.05,
   'n_controle': nan,
   'descricao': 'CONDICIONADOR DE AR SPLIT INVERTER - HI-WALL COM ROTAÇÃO VARIÁVEL \x96 CAPACIDADE 11.500 A 12.000 BTU/H, FRIO, COM INSTALAÇÃO INCLUSA - CONDICIONADOR DE AR SPLIT, 12.000 BTUS, MARCA ELECTROLUX, MODELO BI12F, FRIO, COR BRANCA, 220V. ACOMPANHA CONTROLE REMOTO, MANUAL, ACESSÓRIOS E UNIDADE CONDENSADORA.',
   'peso': '',
   'n_serie': nan,
   'cod_barras': 322563,
   'incorporacao': '2014-05-15T00:00:00+00:00',
   'conservacao': 'EM USO',
   'transf_local': '2017-11-17T00:00:00+00:00',
   'situacao': 'Ativo'},
  'responsavel': 'Rafael dos Reis A

In [56]:

def process_all_ares():
    for ar in [x for x in ares_condicionados if type(x['key']) == int]:
        item_original, lista_partes, lista_volumes = process_item(ar) 
        yield item_original, lista_partes, lista_volumes

In [40]:
import json
lista = [{'original': a, 'partes': b, 'volumes': c} for (a,b,c) in process_all_ares()]
with open("arescondicionares.json", "w") as fp:
    json.dump(lista, fp, indent=2, ensure_ascii=False, default=str) 

In [41]:
ares107 = [x for x in ares_condicionados if x["origem"] == "A107"]

In [57]:
_items = {}
_volumes = {}
for item_original, lista_partes, lista_volumes in process_all_ares():
    _items[item_original["key"]] = item_original
    for parte in lista_partes:
        _items[parte["key"]] = parte
    for volume in lista_volumes:
        _volumes[volume["codigo"]] = volume
        

In [59]:
def typer(dicionario):
    if isinstance(dicionario, dict):
        return {k: typer(v) for (k,v) in dicionario.items()}
    else:
        return type(dicionario)

for key, value in _items.items(): 
    #print(typer(value))
    db.document(f"items/{key}").set(value)
    

In [53]:

for key, value in _items.items(): 
    print(typer(value))
    

{'origem': <class 'str'>, 'edificio': <class 'str'>, 'short_descricao': <class 'str'>, 'key': <class 'int'>, 'tipo': <class 'str'>, 'ambiente': <class 'google.cloud.firestore_v1.document.DocumentReference'>, 'meta': {'volume': <class 'NoneType'>, 'updated': <class 'datetime.datetime'>, 'inteiro': <class 'bool'>, 'volumado': <class 'bool'>, 'partes': <class 'list'>}, 'detalhes': {'patrimonio': <class 'int'>, 'medidas': <class 'str'>, 'valor': <class 'float'>, 'n_controle': <class 'float'>, 'descricao': <class 'str'>, 'peso': <class 'str'>, 'n_serie': <class 'float'>, 'cod_barras': <class 'int'>, 'incorporacao': <class 'NoneType'>, 'conservacao': <class 'str'>, 'transf_local': <class 'NoneType'>, 'situacao': <class 'str'>}, 'responsavel': <class 'str'>}
{'origem': <class 'str'>, 'edificio': <class 'str'>, 'short_descricao': <class 'str'>, 'key': <class 'str'>, 'tipo': <class 'str'>, 'ambiente': <class 'google.cloud.firestore_v1.document.DocumentReference'>, 'meta': {'volume': <class 'Non

In [62]:

for key, value in _volumes.items(): 
    #print(typer(value))
    db.document(f"volumes/{key}").set(value)

In [61]:
_volumes

{'V227811E': {'medidas': {'a': 55, 'l': 78, 'c': 32},
  'categoria': 'Ar Condicionado',
  'peso': 29,
  'items': [<google.cloud.firestore_v1.document.DocumentReference at 0x7fa380249f30>],
  'observacao': 'Criado pelo sistema',
  'data_criacao': datetime.datetime(2023, 12, 14, 16, 8, 38, 436988),
  'embalagem': {'caixa': None, 'platico_bolha': None, 'enchimento': True},
  'responsavel': <google.cloud.firestore_v1.document.DocumentReference at 0x7fa38265ff10>,
  'origem': <google.cloud.firestore_v1.document.DocumentReference at 0x7fa38024a110>,
  'destino': <google.cloud.firestore_v1.document.DocumentReference at 0x7fa38024a860>,
  'propriedades': ['Frágil', 'Proibido Empilhar'],
  'codigo': 'V227811E',
  'servicos': ['Montagem e Desmontagem', 'Especializados'],
  'status': ['Criado', 'Para Desmontagem'],
  'localizacao_atual': <google.cloud.firestore_v1.document.DocumentReference at 0x7fa38024a110>,
  'deleted': False},
 'V227811I': {'medidas': {'a': 29, 'l': 86, 'c': 20},
  'categoria

In [3]:
query = db.collection("volumes").where(filter=FieldFilter("categoria", '==', 'Ar Condicionado')).where(filter=FieldFilter("destino", '==', db.document("ambientes-norte/None")))

In [5]:
ERRADOS = []
for volume in query.stream():
    ERRADOS.append(volume.to_dict())

In [9]:
CORRECT_DESTINOS = {
    "A004": "B018",
    "A201": "A029.12",
    "A211": "A023",
    "B003": "A050",
    "B025": "A029.10",
    "B105": "A131",
    "B121": "B118",
    "C204.1": "B213",
    "C306": "B206",
    "C401": "B315"
}

CORRECT_DESTINOS = {db.document(f"ambientes/{k}"): db.document(f"ambientes-norte/{v}") for (k,v) in CORRECT_DESTINOS.items()}
 

In [14]:
for x in ERRADOS:
    #x["destino"] = CORRECT_DESTINOS[x["origem"]]
    db.document(f"volumes/{x['codigo']}").update({"destino": CORRECT_DESTINOS[x["origem"]]})


In [19]:
query = db.collection("volumes").where(filter=FieldFilter("categoria", '==', 'Ar Condicionado')).where(filter=FieldFilter("responsavel", '==', db.document("usuarios/flavio.jerez@ufsc.br")))

for volume in query.stream():
    volume_dict = volume.to_dict()
    print(volume_dict["codigo"])

9QIBwQrzFH8tv4jqgcCH
EtYeOVWnAOYVU22yxQDe
IwPhkEpQvZ7sLddFBxBa
IywY5B16BE2xwaxWDJMM
LNoZHj8FoCnKtNgRlGe7
OJvzObxroQymXkbkJB2F
RFrJISyvTMSaJ9UHbW0l
V322577E
V322577I
b1XiR9nalvYzn367yHwL
b4YJfBzStmwtGTqW8CJJ
cOumWP8VDfnJoXgR03QF
llRFzySnqnlCimbA9uDc
xzxir6fpIojM4yPBZWK3
